In [32]:
import psycopg2
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split,cross_val_score
import datetime as dt

In [33]:
dbname = 'aleksandargeorgiev'
user = 'aleksandargeorgiev'
password = 'none'
host = 'localhost'  # or your PostgreSQL server IP address
port = '5432'       # default PostgreSQL port

conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

# Create a cursor object using the connection
cursor = conn.cursor()



In [34]:
query='''INSERT INTO patients(check_in, check_out)
SELECT
  make_time(check_in_hour, check_in_minute, check_in_second) as check_in,
  make_time(check_in_hour + (random() * (2 - 1))::int, check_in_minute + (random() * (30 - 0))::int, check_in_second + (random() * (30 - 0))::int) as check_out
FROM (
  SELECT
    floor(random() * (23 - 1))::int + 1 as check_in_hour, -- Random hour between 1 and 22
    floor(random() * (30 - 0))::int as check_in_minute,   -- Random minute between 0 and 59
    floor(random() * (30 - 0))::int as check_in_second    -- Random second between 0 and 59
  FROM
    generate_series(1, 20000) AS event_id
) AS times;'''

cursor.execute(query)

conn.commit()


In [35]:
cursor.execute('''SELECT * FROM patients''')

rows=cursor.fetchall()
'''for row in rows:
    print(row)'''



'for row in rows:\n    print(row)'

In [36]:
df = pd.DataFrame(rows, columns=["id","checkin","checkout"])
df.head()

,id,checkin,checkout
0,2061,07:24:20,07:25:46
1,2062,05:10:04,05:38:20
2,2063,22:09:18,22:15:46
3,2064,16:04:27,17:22:51
4,2065,11:20:13,12:32:18


In [37]:
df["checkin_sec"] = df["checkin"].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)
df["checkout_sec"] = df["checkout"].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)

x=np.array(df["checkin_sec"]).reshape(-1,1)
y=np.array(df["checkout_sec"]).reshape(-1,1)

#print(df["checkin_sec"].shape)
#print(df["checkout_sec"].shape)
print(x.shape)
print(y.shape)

(440000, 1)
(440000, 1)


test-train split and error-list

In [38]:
cross_score={}
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,shuffle=True,random_state=42)


In [39]:
lin_reg=linear_model.LinearRegression().fit(x_train,y_train)
cross_score["lin_reg"]=cross_val_score(lin_reg,x,y,cv=10)


In [40]:
lin_lasso_1=linear_model.Lasso(alpha=0.10).fit(x_train,y_train)
lin_lasso_01=linear_model.Lasso(alpha=0.01).fit(x_train,y_train)
lin_lasso_15=linear_model.Lasso(alpha=0.15).fit(x_train,y_train)
lin_lasso_2=linear_model.Lasso(alpha=0.20).fit(x_train,y_train)

names=[lin_lasso_1,lin_lasso_01,lin_lasso_15,lin_lasso_2]

for name in names:
    cross_score[name]=cross_val_score(name,x,y,cv=10)


In [41]:
lin_ridge_1=linear_model.Ridge(alpha=0.10).fit(x_train,y_train)
lin_ridge_01=linear_model.Ridge(alpha=0.01).fit(x_train,y_train)
lin_ridge_15=linear_model.Ridge(alpha=0.15).fit(x_train,y_train)
lin_ridge_2=linear_model.Ridge(alpha=0.20).fit(x_train,y_train)

names=[lin_ridge_1,lin_ridge_01,lin_ridge_15,lin_ridge_2]

for name in names:
    cross_score[name]=cross_val_score(name,x,y,cv=10)



maybe do a NN but may be overkill
-lets see the results


In [42]:
for mod,score in cross_score.items():
    cross_score[mod]=np.mean(score)

for mod,score in cross_score.items():
    print(f"{str(mod):<20}"+" has right in: "+f"{(score)}"+" %")



lin_reg              has right in: 0.9933160699425093 %
Lasso(alpha=0.1)     has right in: 0.9933160699425091 %
Lasso(alpha=0.01)    has right in: 0.9933160699425091 %
Lasso(alpha=0.15)    has right in: 0.9933160699425093 %
Lasso(alpha=0.2)     has right in: 0.9933160699425091 %
Ridge(alpha=0.1)     has right in: 0.9933160699425093 %
Ridge(alpha=0.01)    has right in: 0.9933160699425093 %
Ridge(alpha=0.15)    has right in: 0.9933160699425093 %
Ridge(alpha=0.2)     has right in: 0.9933160699425093 %


use those models to predict:

In [44]:
my_time=[5:10:24].dt.time

SyntaxError: invalid syntax (3159555883.py, line 1)